<h1>Capstone - Week 3 - Final</h1>

<h2>Import Our Libraries</h2>

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

<h2>Request the URL, convert it to BS, and pull out the table</h2>

In [ ]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
pageMarkup = BeautifulSoup(page.read(),"lxml")

postalCodesTable = pageMarkup.find_all('table')[0]

<h2>Convert the table to a dataframe </h2>

In [3]:
df = pd.read_html(str(postalCodesTable))
df = df[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<h2>Clean up the 'Not assigned' fields</h2>

In [4]:
df = df[(df[[0,1,2]] != 'Not assigned').all(axis=1)]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


<h2>Loop through the dataframe to group rows on borough</h2>

In [39]:
newDf = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

oldP = ""
oldB = ""
oldN = ""

newIndex=0

for index, row in df.iterrows():
    
    if index != 0:
        oldP = postcode
        oldB = borough
        oldN = neighborhood
        
    postcode = row[0]
    borough = row[1]
    neighborhood = row[2]
    
    if borough == oldB:
        neighborhood = oldN + ", " + neighborhood
    elif borough != oldB:
        newDf.loc[newIndex] = [oldP,oldB,oldN]
        newIndex += 1
        
    if index == len(df.index) - 1:
        newDf.loc[newIndex] = [postcode,borough,neighborhood]
        
newDf.drop(newDf.index[:2], inplace=True)

newDf.head()


,Postcode,Borough,Neighborhood
2,M4A,North York,"Parkwoods, Victoria Village"
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"


<h2>Show the dimensions of the final dataframe</h2>

In [40]:
newDf.shape

(83, 3)

<h2>Setup GeoCoder</h2>

In [46]:
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [79]:
geoDf = pd.DataFrame(columns=['Postcode','Borough','Neighborhood','Latitude','Longitude'])

newIndex = 0

for index, row in newDf.iterrows():

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google(row['Postcode'] + ', Toronto, Ontario')
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    geoDf.loc[newIndex] = [row[0],row[1],row[2],latitude,longitude]
    
    newIndex += 1
    
print("done")

done


In [80]:
newDf.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M4A,North York,"Parkwoods, Victoria Village",43.725882,-79.315572
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
